In [29]:
import os
import importlib
import pickle
from src import selenbeau
import urllib.parse

In [12]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_people_links(list_url):
    base_url = "https://en.wikipedia.org"
    response = requests.get(list_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    links = set()
    for li in soup.select('li a[href^="/wiki/"]'):
        href = li.get('href')
        # Filter out non-article links (skip files, categories, help pages, etc.)
        if not any(href.startswith(f"/wiki/{ns}:") for ns in ['Help', 'File', 'Category', 'Wikipedia', 'Template', 'Portal', 'Talk']):
            full_url = urljoin(base_url, href)
            links.add(full_url)

    return list(links)

In [2]:
def get_page_name(url):
    page_name = url.replace('https://en.wikipedia.org/wiki/','')
    page_name = page_name.replace('_',' ')
    page_name = urllib.parse.unquote(page_name)
    if ':' in page_name:
        return None
    return page_name

In [3]:
try:
    with open(load_path, 'rb') as f:
        people_dob = pickle.load(f)
    errors = list()
except:
    people_dob = dict()
    errors = list()

In [39]:
def process_url(url):
    if url in people_dob:
        return None
    tracer = {'url': url}
    page_name = get_page_name(url)
    if page_name is None:
        return None
    tracer['page_name'] = page_name
    try:
        page_id = selenbeau.get_wiki_id_from_page_title(page_title=page_name, open_browser=False)
        tracer['page_id'] = page_id
        dob = selenbeau.get_dob_from_wikidata(page_id)
        tracer['dob'] = dob
        return tracer
    except Exception:
        errors.append(tracer)
        return None

In [5]:
core_urls = [
    'https://en.wikipedia.org/wiki/Party_and_state_leaders',
    'https://en.wikipedia.org/wiki/List_of_Chinese_by_net_worth'
]

In [8]:
china_people_dob_path = os.path.join('data', 'outputs', 'walk_wiki', 'china_people_dob.pickle')
processed_core_urls_path = os.path.join('data', 'outputs', 'walk_wiki', 'core_urls_processed.pickle')

In [10]:
try:
    with open(china_people_dob_path, 'rb') as f:
        people_dob = pickle.load(f)
except:
    people_dob = dict()

try:
    with open(processed_core_urls_path, 'rb') as f:
        core_urls_processed = pickle.load(f)
except:
    core_urls_processed = list()

In [34]:
importlib.reload(selenbeau)

<module 'src.selenbeau' from 'C:\\Users\\muhak\\prj\\dev-ds\\prj-web-automation\\src\\selenbeau.py'>

In [ ]:
for core_url in core_urls:
    if core_url in core_urls_processed:
        continue
    inner_urls = extract_people_links(core_url)

    for url in inner_urls:
        data = process_url(url)
        if data:
            people_dob[url] = data
            with open(china_people_dob_path, 'wb') as f:
                pickle.dump(people_dob, f)

    core_urls_processed.append(core_url)        
    with open(processed_core_urls_path, 'wb') as f:
        pickle.dump(core_urls_processed, f)